# Helper functions

In [1]:
import numpy as np

In [2]:
# From Xavier
def strToBits(string):
    res = []
    byte_string = string.encode('utf-8')
    for b in byte_string:
        bit_array = bin(b)[2:]
        bit_array = '00000000'[len(bit_array):] + bit_array
        res.extend(bit_array)
    return res

def stringToChannelInput(string):
    bits = np.array(strToBits(string), dtype='int64')
    return 2*bits - 1

def channelOutputToString(channelOutput):
    bits = ((channelOutput+1)/2).astype('int64').tolist()
    byte_string = ""
    for char_index in range(len(bits)//8):
        bit_list = bits[char_index*8:(char_index+1)*8]
        byte = chr(int(''.join([str(bit) for bit in bit_list]), 2))
        byte_string += byte
    return byte_string

In [3]:
# From handout
def channel(chanInput):
    chanInput = np.clip(chanInput,-1,1)
    erasedIndex = np.random.randint(3) 
    chanInput[erasedIndex:len(chanInput):3] = 0
    return chanInput + np.sqrt(10)*np.random.randn(len(chanInput))

In [4]:
def channelWoNoise(chanInput, erasedIndex=None):
    """
    Emulates communication channel without noise.
    erasedIndex can be used to specify H
    """
    chanInput = np.clip(chanInput,-1,1)
    erasedIndex = erasedIndex
    if (erasedIndex == None):
        erasedIndex = np.random.randint(3)
    chanInput[erasedIndex:len(chanInput):3] = 0
    return chanInput

In [5]:
import string
import random
import io

In [6]:
def generateTestFile(characters=80, filename="scratch"):
    """
    Function to generate a file containing an utf-8 encoded string.
    Returns the generated text.
    """
    text = ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation, k=characters))
    with io.open(filename+".txt", "w", encoding='utf8') as f:
        f.write(text)
    return text

In [7]:
def readTestFile(filename="scratch"):
    """
    Function to read a text file as channel input.
    Returns the file text in the channel input format.
    """
    text = ''
    with io.open(filename+".txt", encoding='utf8') as f:
        text = f.read()
    return stringToChannelInput(text)

In [30]:
signal_set = np.array([[1, 1, 1], [1, -1, -1], [-1, 1, -1], [-1, -1, 1]])

def encodeChannelInput(chan_input):
    """
    Makes channel input ready for transmission
    """
    # map each bit to two signals appended: 1 is signals 0 and 2, -1 is signals 1 and 3 ()
    def mapBitToSignals(bits):
        if (np.array_equal(bits, [1, 1])):
            return signal_set[0]
        elif (np.array_equal(bits, [1, -1])):
            return signal_set[1]
        elif (np.array_equal(bits, [-1, 1])):
            return signal_set[2]
        elif (np.array_equal(bits, [-1, -1])):
            return signal_set[3]

    chan_input = np.split(chan_input, len(chan_input)/2)
    chan_input = np.array([mapBitToSignals(bit) for bit in chan_input])

    # flatten result
    chan_input = chan_input.flatten()
    return chan_input

In [176]:
def decodeChannelOutputWoNoise(chan_output, erasedIndex=None):
    """
    Decodes channel output where H is known
    """
    # split output
    chan_output = np.split(chan_output, len(chan_output)/len(signal_set[0]))

    # decide on H -> when to compute? for each block? once?
    def decide_on_H(bits):
        return np.argmin(np.array([x**2 for x in bits]))

    H = np.array([decide_on_H(bits) for bits in chan_output]) 
    erasedIndex = np.bincount(H).argmax()

    # decide on index of codeword
    def decoder_H0(input):
        if (input[1] > 0 and input[2] > 0):
            return 0
        elif (input[1] > 0 and input[2] < 0):
            return 2
        elif (input[1] < 0 and input[2] < 0):
            return 1
        elif (input[1] < 0 and input[2] > 0):
            return 3

    def decoder_H1(input):
        if (input[0] > 0 and input[2] > 0):
            return 0
        elif (input[0] > 0 and input[2] < 0):
            return 1
        elif (input[0] < 0 and input[2] < 0):
            return 2
        elif (input[0] < 0 and input[2] > 0):
            return 3

    def decoder_H2(input):
        if (input[0] > 0 and input[1] > 0):
            return 0
        elif (input[0] > 0 and input[1] < 0):
            return 1
        elif (input[0] < 0 and input[1] < 0):
            return 3
        elif (input[0] < 0 and input[1] > 0):
            return 2

    res = np.array([])
    if (erasedIndex == 0):
        res = np.array([decoder_H0(y) for y in chan_output])
    elif (erasedIndex == 1):
        res = np.array([decoder_H1(y) for y in chan_output])
    elif (erasedIndex == 2):
        res = np.array([decoder_H2(y) for y in chan_output])

    # reconstitute channel input
    def retrieveInput(index):
        return signal_set[index][:2]

    res = np.array([retrieveInput(i) for i in res])
    
    res = res.flatten()

    return channelOutputToString(res)

In [184]:
text_in = generateTestFile(characters=80)
print("input:\t" + text_in)
chan_input = encodeChannelInput(readTestFile())
chan_output = channel(chan_input)
text_out = decodeChannelOutputWoNoise(chan_output)
print("output:\t" + text_out)
diff = sum(text_in[i] != text_out[i] for i in range(len(text_out)))
print("diff:\t" + str(diff))

input:	f+BiqjU2qYfeU-qylT#H2'>C*L$Vh6sKm'mt%GekyHNK^?S<-+WG0tBP[kYB4ev]IF#xB8Gs~,;X4zk]
output:	~#ôI)ì-òä¹Kýzµ/eÇ7þlâk)0Êµ¤Òhwâ)èd÷þx´uVú1LE]ËÊEwÊYçâ_mú6;»
diff:	79
